# Stable Diffusion Objectively Succeeds at Copycatting Specific Artists’ Styles

TL;DR: We compiled a set of 62 professional digital artists, each with a large amount of copyrighted work online. We found that when Stable Diffusion is prompted to imitate the artists, the artists were classified from the image successfully an average of 52/62 times (82.74%) and at best 58/62 times (93.54%).

Carl Guo,* MIT, carlguo@mit.edu 

Shreya Mogulothu,* MIT, smog@mit.edu 

Chinmay Deshpande,* Harvard College, cdeshpande@college.harvard.edu

Zachary Marinov,* MIT, zmarinov@mit.edu

Stephen Casper,* MIT, scasper@mit.edu 

*Equal contribution among all authors

For a non-technical writeup of the results, please click [here](https://github.com/thestephencasper/sd_cycle_consistency/tree/main).

## Setting up the experiment environment

In [ ]:
%%capture

# install libraries
!pip install -q transformers
!pip install -q diffusers
!pip install -q accelerate

# import libraries
import matplotlib.pyplot as plt
from diffusers import StableDiffusionPipeline
import torch
import accelerate
from google.colab import files
from PIL import Image
from transformers import CLIPProcessor, CLIPModel

In [ ]:
%%capture

# set up stable diffusion
# download diffusion
model_id = "runwayml/stable-diffusion-v1-5"
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)

# gets rid of content warnings for more data; can remove
pipe.safety_checker = lambda images, clip_input="": (images, False)
pipe = pipe.to("cuda")

# download clip
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

## Choosing parameters for experiment

In this section, we provide two datasets of artists, one with 62 artists selected through a set of criteria detailed in the writeup and another with 250 artists that have the highest amount of images in the LAION-aesthetics dataset. 

We set the prompt to be "Artwork from [artist's name]". `N` determines how many times the experiment would run. These parameters can be customized for further experiments if people want to play with the notebook. 


In [ ]:
# 62 artists
# artists = ['Raphael Lacoste', 'James Jean', 'Andy Kehoe', 'Jeremy Mann', 'Marc Simonetti', 'Ismail Inceoglu', 'Michael Cheval', 'Simon Stalenhag', 'Alex Ross', 'Brian Froud', 'Iain Faulkner', 'Ted Naismith', 'Artgerm', 'Mark Keathley', 'Jeremy Geddes', 'Janek Sedlar', 'Peter Mohrbacher', 'Naoto Hattori', 'Banksy', 'Greg Hildebrandt', 'Nathan Wirth', 'Yuumei', 'Brooke Shaden', 'Atey Ghailan', 'Dorina Costras', 'Filip Hodas', 'Erin Hanson', 'Botero', 'Ai Weiwei', 'Audrey Kawasaki', 'Odd Nerdrum', 'Ian McQue', 'Fabian Perez', 'David Hockney', 'Alex Andreev', 'Steve Henderson', 'Ryan McGinley', 'Peter Gric', 'Kehinde Wiley', 'Stephan Martiniere', 'Alex Grey', 'Catrin Welz-Stein', 'Anna Razumovskaya', 'RHADS', 'Amanda Clark', 'Beeple', 'Sparth', 'Michael Whelan', 'James Gilleard', 'Mikko Lagerstedt', 'Carne Griffiths', 'John Berkey', 'Alexander Jansson', 'Michael Parkes', 'Duy Huynh', 'John Howe', 'Tibor Nagy', 'Shepard Fairey', 'Charlie Bowater', 'Kevin Sloan', 'Andrea Kowch', 'Jimmy Lawlor']

# 250 artists version 
# artists = ['Thomas Kinkade', 'Vincent Van Gogh', 'Leonid Afremov', 'Claude Monet', 'Edward Hopper', 'Norman Rockwell', 'William-Adolphe Bouguereau', 'Albert Bierstadt', 'John Singer Sargent', 'Pierre-Auguste Renoir', 'Frida Kahlo', 'John William Waterhouse', 'Winslow Homer', 'Walt Disney', 'Phil Koch', 'Thomas Moran', 'Paul C√©zanne', 'Camille Pissarro', 'Erin Hanson', 'Thomas Cole', 'Raphael', 'Steve Henderson', 'Pablo Picasso', 'Caspar David Friedrich', 'Ansel Adams', 'Diego Rivera', 'Steve McCurry', 'Bob Ross', 'John Atkinson Grimshaw', 'Rob Gonsalves', 'Paul Gauguin', 'James Tissot', 'Edouard Manet', 'Alphonse Mucha', 'Alfred Sisley', 'Fabian Perez', 'Gustave Courbet', 'Zaha Hadid', 'Jean-L√©on G√©r√¥me', 'Carl Larsson', 'Mary Cassatt', 'Sandro Botticelli', 'Daniel Ridgway Knight', 'Joaqu√≠n Sorolla', 'Andy Warhol', 'Kehinde Wiley', 'Alfred Eisenstaedt', 'Gustav Klimt', 'Dante Gabriel Rossetti', 'Tom Thomson', 'Edgar Degas', 'Utagawa Hiroshige', 'Camille Corot', 'Edward Steichen', 'David Hockney', 'Ivan Aivazovsky', 'Josephine Wall', 'Peter Paul Rubens', 'Henri Rousseau', 'Edward Burne-Jones', 'Pixar', 'Alexander McQueen', 'Anders Zorn', 'Jean Auguste Dominique Ingres', 'Franz Xaver Winterhalter', 'Katsushika Hokusai', 'John Constable', 'Canaletto', 'Gordon Parks', 'Shepard Fairey', 'George Inness', 'Anthony van Dyck', 'Vivian Maier', 'Catrin Welz-Stein', 'Lawren Harris', 'Salvador Dali', 'David Bowie', 'Agnes Cecile', 'Titian', 'Martin Johnson Heade', 'Scott Naismith', 'William Morris', 'Berthe Morisot', 'Vladimir Kush', 'William Holman Hunt', 'Edvard Munch', 'Joseph Mallord William Turner', 'Gustave Dor√©', 'Thomas Eakins', 'Ilya Repin', 'Amedeo Modigliani', 'Johannes Vermeer', 'Eyvind Earle', 'Ivan Shishkin', 'Rembrandt Van Rijn', 'Gil Elvgren', 'Nicholas Roerich', 'Henri Matisse', 'Thomas Gainsborough', 'Artgerm', 'Grant Wood', 'Studio Ghibli', 'Jeremy Mann', 'Mark Keathley', 'Maxfield Parrish', 'Andrew Wyeth', 'RHADS', 'David Lynch', 'Frederic Remington', 'Jan Van Eyck', 'Mikko Lagerstedt', 'Banksy', 'Michael Cheval', 'Anna Razumovskaya', 'Jean-Fran√ßois Millet', 'Thomas W Schaller', 'Charlie Bowater', 'El Greco', 'Paolo Roversi', 'Carne Griffiths', 'Man Ray', 'Andrew Macara', 'August Sander', 'Evelyn De Morgan', 'Sally Mann', 'William Blake', 'Oleg Oprisco', 'Yuumei', 'Helmut Newton', 'Henry Ossawa Tanner', 'Asher Brown Durand', 'teamLab', 'August Macke', 'Armand Guillaumin', 'Terry Redlin', 'Antoine Blanchard', 'Anna Ancher', 'Ohara Koson', 'Walter Langley', 'Yayoi Kusama', 'Stan Lee', 'Chuck Close', 'Albert Edelfelt', 'Mark Seliger', 'Eugene Delacroix', 'John Lavery', 'Theo van Rysselberghe', 'Marc Chagall', 'Rolf Armstrong', 'Brent Heighton', 'A.J.Casson', 'Egon Schiele', 'Maximilien Luce', 'Georges Seurat', 'Arthur Hughes', 'George Frederic Watts', 'Anton Mauve', 'Lucian Freud', 'Jessie Willcox Smith', 'Leonardo Da Vinci', 'Edward John Poynter', 'Brooke Shaden', 'J.M.W. Turner', 'Wassily Kandinsky', 'Wes Anderson', 'Jean-Honor√© Fragonard', 'Amanda Clark', 'Tom Roberts', 'Antonello da Messina', 'Hayao Miyazaki', 'Makoto Shinkai', 'Alfred Stevens', 'Slim Aarons', 'Albert Lynch', 'Andre Kohn', 'Daniel Garber', 'Jacek Yerka', 'Beatrix Potter', 'Rene Magritte', "Georgia O'Keeffe", 'Isaac Levitan', 'Frank Lloyd Wright', 'Gustave Moreau', 'Ai Weiwei', 'Ford Madox Brown', 'Tim Burton', 'Alfred Cheney Johnston', 'Duy Huynh', 'Michael Parkes', 'Tintoretto', 'Archibald Thorburn', 'Audrey Kawasaki', 'George Lucas', 'Arthur Streeton', 'Albrecht Durer', 'Andrea Kowch', 'Dorina Costras', 'Alex Ross', 'Hasui Kawase', 'Lucas Cranach the Elder', 'Antonio Mora', 'Briton Rivi√®re', 'Mandy Disher', 'Henri-Edmond Cross', 'Auguste Toulmouche', 'Hubert Robert', 'Syd Mead', 'Alyssa Monks', 'Carl Spitzweg', 'Edward Lear', 'Ralph McQuarrie', 'Sailor Moon', 'Simon Stalenhag', 'Edward Robert Hughes', 'Jules Bastien-Lepage', 'Richard S. Johnson', 'Rockwell Kent', 'Sparth', 'Arnold Bocklin', 'Arnold B√∂cklin', 'Lovis Corinth', 'Robert Hagan', 'Gregory Crewdson', 'Abbott Handerson Thayer', 'Thomas Benjamin Kennington', 'Gilbert Stuart', 'Louis Comfort Tiffany', 'Raphael Lacoste', 'Janek Sedlar', 'Jean Marc Nattier', 'Sherree Valentine Daines', 'Alexander Jansson', 'James Turrell', 'Alex Grey', 'Henri De Toulouse Lautrec', 'Anton Pieck', 'Andrew Atroshenko', 'Ramon Casas', 'Andy Kehoe', 'Andreas Achenbach', 'Eric Zener', 'H.P. Lovecraft', 'Kunisada', 'Jimmy Lawlor', 'Quentin Tarantino', 'Marianne North', 'Vivienne Westwood', 'Tom Bagshaw', 'Jeremy Lipking', 'John Martin']

# 62 random names for control from https://randomwordgenerator.com/name.php
# artists = ['Ariel Case', 'Wilfredo Flynn', 'Madeleine Tapia', 'Dion Mason', 'Aimee Donovan', 'Tim Burton', 'Gena Levy', 'Eleanor Monroe', 'Faye Mccarty', 'Jospeh Trevino', 'Leandro Hoover', 'Cary Haney', 'Hannah Carney', 'Claire Blackburn', 'Rodrick Carter', 'Alphonse Kelle', 'Zane Ray', 'Kristi Gregory', 'Leonor Garrett', 'Sergio Randall', 'Garfield Ingram', 'Vanessa Christian', 'Morton Burch', 'Abby Stokes', 'Marion Delacruz', 'Guillermo Chavez', 'Sid Quinn', 'Maynard Wilson', 'Jan Perez', 'Mose Weber', 'Tessa Sanchez', 'Dorsey Robbins', 'Maryellen Davies', 'Dario Hardy', 'Maxine Tucker', 'Amalia Velazquez', 'Ron Padilla', 'Margo Pace', 'Jorge Scott', 'Brett Pham', 'Bonita Good', 'Sara York', 'Randolph Stevens', 'Jefferson Rubio', 'Clair Glenn', 'Sang Jimenez', 'Blanche Zimmerman', 'Kristina Hampton', 'Jamal Cherry', 'Fanny Hunter', 'Walton Schultz', 'Hung Martinez', 'Marcus Phillips', 'Rose Nelson', 'Jerri Jacobs', 'Bertie Turner', 'Hubert Allen', 'Corine Solis', 'Pauline Hobbs', 'Tabitha Wright', 'Catherine Joseph', 'Eva Bullock']

# prompt for each artist (can be customized)
prompts = [f"Artwork from {artist}" for artist in artists]

# run the experiment N times
N = 10

## Running the experiment

In [ ]:
# parameters (most people wont need to worry about thest)
images_per_pipeline = 6  # this helps with RAM issues
cumulative_count = 0 
artists_guessed = set()
# expected artist should be equal to index
expected = torch.arange(len(prompts))
result = [] 

for i in range(N):
  # need to break it up bc even with "accelerate" you still run into memory issues
  images = []
  for index in range(int(len(prompts)/images_per_pipeline)+2): 
      prompt_portion = prompts[images_per_pipeline*index:images_per_pipeline*(index+1)]
      if prompt_portion:
        images += pipe(prompt_portion).images

  # plotting the images
  n_col = 5
  # setting the size; can be adjusted
  plt.figure(figsize=(20, int(len(images) / n_col) * 5))
  # plot each image in correct location with title given by artist name
  for j in range(len(images)):
    plt.subplot(int(len(images) / n_col) + 1, n_col, j + 1)
    plt.imshow(images[j])
    plt.title(artists[j], fontsize=13, fontweight='bold')
    plt.xticks([])
    plt.yticks([])
  plt.savefig(f"sd_imitation_{i}.png")
  plt.show()

  # now let CLIP caption them using the prompts inserted. combine images1-4
  # also change prompts for CLIP captioning
  inputs = processor(text=prompts, images=images, return_tensors="pt", padding=True) # pt means "pytorch"

  # the results
  outputs = model(**inputs)
  logits_per_image = outputs.logits_per_image  # this is the image-text similarity score
  probs = logits_per_image.softmax(dim=1)  # we can take the softmax to get the label probabilities

  # best guess has highest probability
  best_guess = torch.argmax(probs, dim=1)
  artists_guessed.update(set(best_guess.tolist()))
  print(best_guess)
  # count number you get right by seeing where it matches w "expected", the vector we set up earlier
  best_diff = best_guess - expected
  best_count = len(prompts) - torch.count_nonzero(best_diff)
  cumulative_count += len(prompts) - torch.count_nonzero(best_diff)
  top3_guesses = torch.topk(probs, 3, dim=1).indices
  top3_diff = torch.sub(top3_guesses, expected[:, None])
  top3_count = torch.count_nonzero((top3_diff == 0).sum(dim=1))

  # accuracy :) 0-indexing, so we add 1 to round number
  print(f"best guess count: {best_count / len(prompts)}")
  print(f"top 3 guess count: {top3_count / len(prompts)}")
  print(f"average: {cumulative_count / (len(prompts) * (i + 1))}")
  result.append((best_count, top3_count, cumulative_count))

print(f"amount of artists guessed at least once {len(artists_guessed)} out of total {len(prompts)}")
